<a href="https://colab.research.google.com/github/Masty3/Project1/blob/main/OCRkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract
!pip install mysql.connector
#PyMuPDF is imported by using import fitz
!pip install PyMuPDF
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 61.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import mysql.connector
import os
import fitz
import pytesseract
import cv2
import numpy as np

In [ ]:
#find a way to get the exe
#works only by installing Tesseract-OCR locally
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
#connect to database by using the connector
mydb = mysql.connector.connect(
    host="localhost",
    user="rsx",
    password="Jimi70..",
    database="MYSQL"
)

In [ ]:
#create a cursor object to execute the import of a new pdf file and store it into the database
#is an object provided by MySQL connector library, allows the communication with database
mycursor = mydb.cursor()

In [ ]:
#create a table if it not exists yet with the name pdf_text
mycursor.execute("CREATE TABLE IF NOT EXISTS pdf_text (id INT AUTO_INCREMENT PRIMARY KEY, filename VARCHAR(255), text LONGTEXT)")

In [ ]:
#develop a function that extract text from pdf using ocr
#we will initialize an empty string used to store the extracted text from pdf by using text = ""
#then we iterate through each page of the pdf files by page_count
#pix is used to create a pixmap to improve ocr quality
#then we reshape the image to an NumPy array image
#gauss kernel is applied to reduce noise and smoothen the image
def extract_text_from_pdf(filename):
    with fitz.open(filename) as doc:
        text = ""
        for page_number in range(doc.page_count):
            page = doc.load_page(page_number)
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2), alpha=False)
            img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, 3)

            # Preprocessing
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
            _, thresh_img = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

            #this function returns the concatenated text
            text += pytesseract.image_to_string(thresh_img)

        return text

In [ ]:
#function to insert the text from the pdf file into the database
#we are creating a query to insert value filename and text into the pdf_text table
def insert_into_db(filename, text):
    sql = "INSERT INTO pdf_text (filename, text) VALUES (%s, %s)"
    val = (filename, text)
    mycursor.execute(sql, val)
    mydb.commit()

In [ ]:
import collections #needed to store words in a dictionary

In [ ]:
def count_words(text):
    words = text.lower().split()
    return collections.Counter(words)

# Initialize an empty dictionary to store word counts
word_counts = collections.Counter()

In [ ]:
#now we got to ensure that all pdf files within one directory will be processed
folder_path = 'C:\\Users\\rekic\\OneDrive\\Dokumente\\FHWN\\2. Sem\\HC Project 1\\pdffiles'

In [ ]:
for filename in os.listdir(folder_path):
    try:
        file_path = os.path.join(folder_path, filename) #used to construct the full path to an file

        if filename.endswith('.pdf'):
            extracted_text = extract_text_from_pdf(file_path)
        elif filename.endswith('.jpg'):
            extracted_text = extract_text_from_image(file_path)
        else:
            continue
        
        word_counts.update(count_words(extracted_text))

        print(f"Extracted text from {filename}:\n")
        print(extracted_text)
        print("\n" + "-" * 80 + "\n")  # Print a separator line between files
        insert_into_db(filename, extracted_text)
    except Exception as e:
        print(f"Error processing {filename}: {e}")
    
for word, count in word_counts.items():
      print(f"{word}: {count}")

In [ ]:
#close the database connection
mydb.close()